In [3]:
# Standart libraries
import os
import glob

# Third-party libraries
import pandas as pd

# API libraries
import openmeteo_requests
import requests_cache
from retry_requests import retry

# Local libraries
from utils import data_check as ck
from utils import data_cleaning as dc

# Enable auto-reload for modules during development
%load_ext autoreload
%autoreload 2

In [ ]:
#https://historical-forecast-api.open-meteo.com/v1/forecast?latitude=54.7584&longitude=-2.6953&start_date=2019-01-01&end_date=2025-04-12&hourly=direct_radiation,direct_normal_irradiance,terrestrial_radiation,direct_radiation_instant,direct_normal_irradiance_instant,terrestrial_radiation_instant,temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,cloud_cover,cloud_cover_low,cloud_cover_mid,wind_speed_10m,wind_speed_80m,wind_direction_10m,wind_direction_80m,wind_gusts_10m,sunshine_duration,shortwave_radiation,diffuse_radiation,shortwave_radiation_instant,diffuse_radiation_instant,global_tilted_irradiance_instant,global_tilted_irradiance,wind_speed_120m,wind_speed_180m,wind_direction_120m,wind_direction_180m,temperature_80m,temperature_120m,temperature_180m

In [13]:
def get_start_end_dates():
    # Get the list of CSV files
    csv_files = glob.glob('../data/weather/uk/open*.csv')
    csv_file = csv_files[0]

    # Initialize start_date and end_date
    start_date = None
    end_date = None
        
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Check if the DataFrame is empty
    if df.empty:
        print(f"Warning: The file {csv_file} is empty.")
        return None, None

    # Get the last date from the 'date' column
    last_date_str = df['time'].iloc[-1]
    last_date = pd.to_datetime(last_date_str)

    # Add one day to the last date
    start_date = last_date + pd.Timedelta(days=1)

    # Get today's date and subtract 3 days
    today = pd.to_datetime('today')
    end_date = today - pd.Timedelta(days=3)

    return start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')

In [14]:
start_date, end_date = get_start_end_dates()

In [15]:
print(f"Start date: {start_date}")
print(f"End date: {end_date}")

Start date: 2025-04-01
End date: 2025-04-12


In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": 54.7584,
	"longitude": -2.6953,
	"start_date": start_date,
	"end_date": end_date,
	"hourly": ["direct_radiation", "direct_normal_irradiance", "terrestrial_radiation", "direct_radiation_instant", "direct_normal_irradiance_instant", "terrestrial_radiation_instant", "temperature_2m", "relative_humidity_2m", "apparent_temperature", "precipitation", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "wind_speed_10m", "wind_speed_80m", "wind_direction_10m", "wind_direction_80m", "wind_gusts_10m", "sunshine_duration", "shortwave_radiation", "diffuse_radiation", "shortwave_radiation_instant", "diffuse_radiation_instant", "global_tilted_irradiance_instant", "global_tilted_irradiance", "wind_speed_120m", "wind_speed_180m", "wind_direction_120m", "wind_direction_180m", "temperature_80m", "temperature_120m", "temperature_180m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_direct_radiation = hourly.Variables(0).ValuesAsNumpy()
hourly_direct_normal_irradiance = hourly.Variables(1).ValuesAsNumpy()
hourly_terrestrial_radiation = hourly.Variables(2).ValuesAsNumpy()
hourly_direct_radiation_instant = hourly.Variables(3).ValuesAsNumpy()
hourly_direct_normal_irradiance_instant = hourly.Variables(4).ValuesAsNumpy()
hourly_terrestrial_radiation_instant = hourly.Variables(5).ValuesAsNumpy()
hourly_temperature_2m = hourly.Variables(6).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(7).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(8).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(9).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(10).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(11).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(12).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(13).ValuesAsNumpy()
hourly_wind_speed_80m = hourly.Variables(14).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(15).ValuesAsNumpy()
hourly_wind_direction_80m = hourly.Variables(16).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(17).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(18).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(19).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(20).ValuesAsNumpy()
hourly_shortwave_radiation_instant = hourly.Variables(21).ValuesAsNumpy()
hourly_diffuse_radiation_instant = hourly.Variables(22).ValuesAsNumpy()
hourly_global_tilted_irradiance_instant = hourly.Variables(23).ValuesAsNumpy()
hourly_global_tilted_irradiance = hourly.Variables(24).ValuesAsNumpy()
hourly_wind_speed_120m = hourly.Variables(25).ValuesAsNumpy()
hourly_wind_speed_180m = hourly.Variables(26).ValuesAsNumpy()
hourly_wind_direction_120m = hourly.Variables(27).ValuesAsNumpy()
hourly_wind_direction_180m = hourly.Variables(28).ValuesAsNumpy()
hourly_temperature_80m = hourly.Variables(29).ValuesAsNumpy()
hourly_temperature_120m = hourly.Variables(30).ValuesAsNumpy()
hourly_temperature_180m = hourly.Variables(31).ValuesAsNumpy()

hourly_data = {"time": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

#hourly_data["direct_radiation_w/m²"] = hourly_direct_radiation
#hourly_data["direct_normal_irradiance_w/m"] = hourly_direct_normal_irradiance
#hourly_data["terrestrial_radiation_w/m²"] = hourly_terrestrial_radiation
hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant
hourly_data["direct_normal_irradiance_instant"] = hourly_direct_normal_irradiance_instant
hourly_data["terrestrial_radiation_instant"] = hourly_terrestrial_radiation_instant
#hourly_data["temperature_2m_°c"] = hourly_temperature_2m
#hourly_data["relative_humidity_2m_%"] = hourly_relative_humidity_2m
#hourly_data["apparent_temperature_°c"] = hourly_apparent_temperature
#hourly_data["precipitation_mm"] = hourly_precipitation
#hourly_data["cloud_cover_%"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
#hourly_data["wind_speed_10m_km/h"] = hourly_wind_speed_10m
#hourly_data["wind_speed_100m_km/h"] = hourly_wind_speed_80m
#hourly_data["wind_direction_10m_°"] = hourly_wind_direction_10m
#hourly_data["wind_direction_100m_°"] = hourly_wind_direction_80m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["sunshine_duration"] = hourly_sunshine_duration
#hourly_data["shortwave_radiation_w/m²"] = hourly_shortwave_radiation
hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
#hourly_data["diffuse_radiation_w/m²"] = hourly_shortwave_radiation_instant
hourly_data["diffuse_radiation_instant"] = hourly_diffuse_radiation_instant
hourly_data["global_tilted_irradiance_instant"] = hourly_global_tilted_irradiance_instant
#hourly_data["global_tilted_irradiance_w/m²"] = hourly_global_tilted_irradiance
hourly_data["wind_speed_120m"] = hourly_wind_speed_120m
hourly_data["wind_speed_180m"] = hourly_wind_speed_180m
hourly_data["wind_direction_120m"] = hourly_wind_direction_120m
hourly_data["wind_direction_180m"] = hourly_wind_direction_180m
hourly_data["temperature_80m"] = hourly_temperature_80m
hourly_data["temperature_120m"] = hourly_temperature_120m
hourly_data["temperature_180m"] = hourly_temperature_180m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.head()

Coordinates 54.76000213623047°N -2.700000047683716°E
Elevation 71.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s


,date,direct_radiation,direct_normal_irradiance,terrestrial_radiation,direct_radiation_instant,direct_normal_irradiance_instant,terrestrial_radiation_instant,temperature_2m,relative_humidity_2m,apparent_temperature,...,diffuse_radiation_instant,global_tilted_irradiance_instant,global_tilted_irradiance,wind_speed_120m,wind_speed_180m,wind_direction_120m,wind_direction_180m,temperature_80m,temperature_120m,temperature_180m
0,2025-04-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,8.03,83.0,6.396517,...,0.0,0.0,0.0,14.654254,19.386593,152.175995,148.671371,10.73,10.93,11.13
1,2025-04-01 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,7.48,86.0,5.766363,...,0.0,0.0,0.0,17.414474,22.668570,150.255203,147.319458,10.23,10.73,11.03
2,2025-04-01 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,7.13,86.0,5.243436,...,0.0,0.0,0.0,22.065973,27.459293,146.309906,145.684860,10.33,10.83,10.93
3,2025-04-01 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,7.58,81.0,5.329763,...,0.0,0.0,0.0,25.864943,29.655787,145.203903,146.888657,10.53,10.93,10.83
4,2025-04-01 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,7.43,81.0,5.193645,...,0.0,0.0,0.0,23.565569,27.059933,145.581573,146.944168,10.53,10.73,10.63


In [17]:
hourly_dataframe = hourly_dataframe.drop([0])
hourly_dataframe.head()

,date,direct_radiation,direct_normal_irradiance,terrestrial_radiation,direct_radiation_instant,direct_normal_irradiance_instant,terrestrial_radiation_instant,temperature_2m,relative_humidity_2m,apparent_temperature,...,diffuse_radiation_instant,global_tilted_irradiance_instant,global_tilted_irradiance,wind_speed_120m,wind_speed_180m,wind_direction_120m,wind_direction_180m,temperature_80m,temperature_120m,temperature_180m
1,2025-04-01 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,7.48,86.0,5.766363,...,0.0,0.0,0.0,17.414474,22.668570,150.255203,147.319458,10.23,10.73,11.03
2,2025-04-01 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,7.13,86.0,5.243436,...,0.0,0.0,0.0,22.065973,27.459293,146.309906,145.684860,10.33,10.83,10.93
3,2025-04-01 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,7.58,81.0,5.329763,...,0.0,0.0,0.0,25.864943,29.655787,145.203903,146.888657,10.53,10.93,10.83
4,2025-04-01 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,7.43,81.0,5.193645,...,0.0,0.0,0.0,23.565569,27.059933,145.581573,146.944168,10.53,10.73,10.63
5,2025-04-01 05:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,6.73,82.0,4.466058,...,0.0,0.0,0.0,22.288042,26.368616,148.877594,147.828812,10.23,10.43,10.43


In [ ]:
# This data would need to pass for a script that clean all the data in the same formt and then concatenate the data to the csv file

In [50]:
# Get data and convert into dataframe
import pandas as pd
import requests
from urllib import parse

sql_query = '''SELECT COUNT(*) OVER () AS _count, * FROM "b2bde559-3455-4021-b179-dfe60c0337b0" WHERE "SETTLEMENT_DATE" >= '2025-03-24T00:00:00.000Z' AND "SETTLEMENT_DATE" <= '2025-04-24T23:59:59.999Z' ORDER BY "_id" ASC LIMIT 100'''

params = {
'sql'
: sql_query}

try:
    resposne = requests.get(
'https://api.neso.energy/api/3/action/datastore_search_sql'
, params = parse.urlencode(params))
    data = resposne.json()[
"result"
]
    df = pd.DataFrame(data[
"records"
])
    print(df) # Dataframe
except requests.exceptions.RequestException as e:
    print(e.response.text)

    IFA_FLOW    TSD  VIKING_FLOW  GREENLINK_FLOW  IFA2_FLOW  \
0       2001  25925          362            -514        992   
1       1999  26637          364            -514        992   
2       1909  26474          324            -514        992   
3       1902  26030          314            -514        991   
4       1968  25352           25            -514        947   
..       ...    ...          ...             ...        ...   
91      2002  32468         1243            -514        992   
92      2001  31031         1410            -514        992   
93      2001  29169         1412            -514        992   
94      2001  27696         1019            -514        992   
95      2001  26858          992            -514        992   

    EMBEDDED_WIND_GENERATION     ND  MOYLE_FLOW  NEMO_FLOW  ELECLINK_FLOW  \
0                        853  23427        -367        765            995   
1                        832  24135        -367        768            995   
2           

In [40]:
df.head(48)

,IFA_FLOW,TSD,VIKING_FLOW,GREENLINK_FLOW,IFA2_FLOW,EMBEDDED_WIND_GENERATION,ND,MOYLE_FLOW,NEMO_FLOW,ELECLINK_FLOW,...,SCOTTISH_TRANSFER,_count,NON_BM_STOR,_full_text,SETTLEMENT_PERIOD,EAST_WEST_FLOW,NSL_FLOW,BRITNED_FLOW,_id,EMBEDDED_SOLAR_GENERATION
0,2001,25925,362,-514,992,853,23427,-367,765,995,...,439,96,0,'-03':2 '-24':3 '-367':18 '-462':19 '-514':24 ...,1,-462,997,799,3937,0
1,1999,26637,364,-514,992,832,24135,-367,768,995,...,588,96,0,'-03':2 '-24':3 '-367':18 '-461':19 '-514':24 ...,2,-461,997,808,3938,0
2,1909,26474,324,-514,992,799,23810,-367,820,995,...,721,96,0,'-03':2 '-24':3 '-367':18 '-382':19 '-514':24 ...,3,-382,997,587,3939,0
3,1902,26030,314,-514,991,765,23400,-367,821,993,...,783,96,0,'-03':2 '-24':3 '-313':19 '-367':18 '-514':24 ...,4,-313,997,560,3940,0
4,1968,25352,25,-514,947,759,22679,-367,504,913,...,868,96,0,'-03':2 '-24':3 '-361':19 '-367':18 '-514':24 ...,5,-361,997,148,3941,0
5,1972,25155,-3,-514,945,753,22485,-367,491,909,...,989,96,0,'-03':2 '-24':3 '-3':23 '-360':19 '-367':18 '-...,6,-360,997,147,3942,0
6,2000,25053,-247,-514,989,729,22012,-367,364,989,...,870,96,0,'-03':2 '-223':17 '-24':3 '-247':23 '-271':19 ...,7,-271,997,-223,3943,0
7,2001,25054,-275,-514,992,705,21977,-367,353,995,...,918,96,0,'-03':2 '-24':3 '-244':19 '-257':17 '-275':23 ...,8,-244,997,-257,3944,0
8,2001,25323,-622,-514,992,702,21675,-367,-211,994,...,1312,96,0,'-03':2 '-159':19 '-211':20 '-24':3 '-367':18 ...,9,-159,997,-630,3945,0
9,2001,25473,-657,-514,991,699,21692,-367,-239,994,...,1400,96,0,'-03':2 '-238':19 '-239':20 '-24':3 '-367':18 ...,10,-238,997,-633,3946,0


In [47]:

import json
import requests

url = 'https://api.neso.energy/api/3/action/datastore_search?resource_id=b2bde559-3455-4021-b179-dfe60c0337b0&limit=100'
response = requests.get(url)
data = response.json() 

In [48]:
data

{'help': 'https://api.neso.energy/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'resource_id': 'b2bde559-3455-4021-b179-dfe60c0337b0',
  'fields': [{'type': 'int', 'id': '_id'},
   {'info': {'comment': 'This follows clock change.',
     'description': 'The date the historic outturn occurred.',
     'title': 'Settlement Date',
     'type': 'date',
     'example': '2023-01-01',
     'unit': 'ISO 8601'},
    'type': 'date',
    'id': 'SETTLEMENT_DATE'},
   {'info': {'comment': '',
     'description': 'The half hourly period for the historic outturn occurred. Settlement period 1 runs from 00:00-00:30.',
     'title': 'Settlement Period',
     'type': 'integer',
     'example': '1',
     'unit': ''},
    'type': 'int4',
    'id': 'SETTLEMENT_PERIOD'},
   {'info': {'comment': '',
     'description': 'This is the Great Britain generation requirement and is equivalent to the Initial National Demand Outturn (INDO) and National Demand Forec

In [ ]:
# Extract the field names (the 'id' values)
field_names = [field['id'] for field in data['result']['fields']]


In [57]:
import requests

resource_id = "b2bde559-3455-4021-b179-dfe60c0337b0"

sql = f'''
SELECT 
  "SETTLEMENT_DATE", 
  "SETTLEMENT_PERIOD", 
  "ND", 
  "TSD", 
  "ENGLAND_WALES_DEMAND", 
  "EMBEDDED_WIND_GENERATION", 
  "EMBEDDED_WIND_CAPACITY", 
  "EMBEDDED_SOLAR_GENERATION", 
  "EMBEDDED_SOLAR_CAPACITY", 
  "NON_BM_STOR", 
  "PUMP_STORAGE_PUMPING", 
  "SCOTTISH_TRANSFER", 
  "IFA_FLOW", 
  "IFA2_FLOW", 
  "BRITNED_FLOW", 
  "MOYLE_FLOW", 
  "EAST_WEST_FLOW", 
  "NEMO_FLOW", 
  "NSL_FLOW", 
  "ELECLINK_FLOW", 
  "VIKING_FLOW", 
  "GREENLINK_FLOW"
FROM "{resource_id}" 
ORDER BY "SETTLEMENT_DATE" DESC, "SETTLEMENT_PERIOD" DESC 
LIMIT 48
'''

url = 'https://api.neso.energy/api/3/action/datastore_search_sql'
params = {'sql': sql}

response = requests.get(url, params=params)
data = response.json()


# Print result or check length
#data['result']['records']
df = pd.DataFrame(data['result']['records'])
df.head(48)


,IFA_FLOW,TSD,VIKING_FLOW,GREENLINK_FLOW,IFA2_FLOW,EMBEDDED_WIND_GENERATION,ND,MOYLE_FLOW,NEMO_FLOW,ELECLINK_FLOW,...,SETTLEMENT_DATE,ENGLAND_WALES_DEMAND,EMBEDDED_SOLAR_CAPACITY,SCOTTISH_TRANSFER,NON_BM_STOR,SETTLEMENT_PERIOD,EAST_WEST_FLOW,NSL_FLOW,BRITNED_FLOW,EMBEDDED_SOLAR_GENERATION
0,2001,26858,992,-514,992,527,24751,-452,987,994,...,2025-03-25,22487,18720,947,0,48,-530,997,1038,0
1,2001,27696,1019,-514,992,518,25594,-452,984,995,...,2025-03-25,23257,18720,876,0,47,-529,997,1031,0
2,2001,29169,1412,-514,992,508,27067,-451,883,994,...,2025-03-25,24660,18720,617,0,46,-530,997,850,0
3,2001,31031,1410,-514,992,542,28929,-451,854,984,...,2025-03-25,26378,18720,576,0,45,-530,997,852,0
4,2002,32468,1243,-514,992,576,30367,-452,260,827,...,2025-03-25,27727,18720,646,0,44,-530,997,905,0
5,2002,34227,1219,-514,992,609,32127,-451,256,833,...,2025-03-25,29327,18720,625,0,43,-530,997,893,0
6,2002,35708,969,-523,992,642,33598,-451,366,990,...,2025-03-25,30667,18720,660,0,42,-530,997,577,0
7,2002,36730,960,-514,992,669,34630,-450,358,989,...,2025-03-25,31634,18720,1051,0,41,-530,997,575,0
8,2003,37610,940,-514,991,695,35508,-450,134,897,...,2025-03-25,32511,18720,1423,0,40,-530,997,535,0
9,2000,38364,942,-514,977,768,36259,-450,122,882,...,2025-03-25,33179,18720,1490,0,39,-530,997,465,0


In [58]:
import pandas as pd

url = "https://api.neso.energy/dataset/88313ae5-94e4-4ddc-a790-593554d8c6b9/resource/f93d1835-75bc-43e5-84ad-12472b180a98/download/df_fuel_ckan.csv"

# Load the CSV directly into a DataFrame
df_fuel = pd.read_csv(url)

# Check the first few rows

df_fuel.head()

,DATETIME,GAS,COAL,NUCLEAR,WIND,WIND_EMB,HYDRO,IMPORTS,BIOMASS,OTHER,...,IMPORTS_perc,BIOMASS_perc,OTHER_perc,SOLAR_perc,STORAGE_perc,GENERATION_perc,LOW_CARBON_perc,ZERO_CARBON_perc,RENEWABLE_perc,FOSSIL_perc
0,2009-01-01T00:00:00,8368.0,15037.0,7099.0,238.0,60.0,246,2524.0,0.0,0.0,...,7.5,0.0,0.0,0.0,0.0,100.0,22.8,24.5,1.6,69.7
1,2009-01-01T00:30:00,8501.0,15095.0,7087.0,219.0,55.0,245,2502.0,0.0,0.0,...,7.4,0.0,0.0,0.0,0.0,100.0,22.6,24.2,1.5,70.0
2,2009-01-01T01:00:00,8475.0,15087.0,7074.0,197.0,49.0,246,2472.0,0.0,0.0,...,7.4,0.0,0.0,0.0,0.0,100.0,22.5,24.2,1.5,70.1
3,2009-01-01T01:30:00,8319.0,15031.0,7064.0,182.0,45.0,246,2445.0,0.0,0.0,...,7.3,0.0,0.0,0.0,0.0,100.0,22.6,24.3,1.4,70.1
4,2009-01-01T02:00:00,8294.0,14999.0,7052.0,167.0,42.0,246,2370.0,0.0,0.0,...,7.1,0.0,0.0,0.0,0.0,100.0,22.6,24.3,1.4,70.2


In [2]:
import requests
import pandas as pd

# Define the API endpoint
url = "https://api.ember-energy.org/v1/options/European%20Wholesale%20Electricity%20Price%20Data/hourly/UK?api_key=bb0bc21f-9e7b-41f1-abe7-bc336ac3b2b9"

# Make the GET request
response = requests.get(url)

# Check the response status
if response.status_code == 200:
    data = response.json()
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Preview the DataFrame
    print(df.head())
else:
    print(f"Failed to retrieve data: {response.status_code}")

Failed to retrieve data: 404


In [ ]:
# Price data in real time is missing

In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": 54.7584,
	"longitude": -2.6953,
	"start_date": "2019-01-01",
	"end_date": "2025-03-30",
	"hourly": ["direct_radiation", "direct_normal_irradiance", "terrestrial_radiation", "temperature_2m", "relative_humidity_2m", "apparent_temperature", "precipitation", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "wind_speed_10m", "wind_speed_80m", "wind_direction_10m", "wind_direction_80m", "shortwave_radiation", "diffuse_radiation", "global_tilted_irradiance", "wind_speed_120m", "wind_speed_180m", "wind_direction_120m", "wind_direction_180m", "temperature_80m", "temperature_120m", "temperature_180m"],
	"timezone": "Europe/London"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_direct_radiation = hourly.Variables(0).ValuesAsNumpy()
hourly_direct_normal_irradiance = hourly.Variables(1).ValuesAsNumpy()
hourly_terrestrial_radiation = hourly.Variables(2).ValuesAsNumpy()
hourly_temperature_2m = hourly.Variables(3).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(4).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(5).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(6).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(7).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(8).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(9).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(10).ValuesAsNumpy()
hourly_wind_speed_80m = hourly.Variables(11).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(12).ValuesAsNumpy()
hourly_wind_direction_80m = hourly.Variables(13).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(14).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(15).ValuesAsNumpy()
hourly_global_tilted_irradiance = hourly.Variables(16).ValuesAsNumpy()
hourly_wind_speed_120m = hourly.Variables(17).ValuesAsNumpy()
hourly_wind_speed_180m = hourly.Variables(18).ValuesAsNumpy()
hourly_wind_direction_120m = hourly.Variables(19).ValuesAsNumpy()
hourly_wind_direction_180m = hourly.Variables(20).ValuesAsNumpy()
hourly_temperature_80m = hourly.Variables(21).ValuesAsNumpy()
hourly_temperature_120m = hourly.Variables(22).ValuesAsNumpy()
hourly_temperature_180m = hourly.Variables(23).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["direct_radiation"] = hourly_direct_radiation
hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
hourly_data["terrestrial_radiation"] = hourly_terrestrial_radiation
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_80m"] = hourly_wind_speed_80m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_direction_80m"] = hourly_wind_direction_80m
hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
hourly_data["wind_speed_120m"] = hourly_wind_speed_120m
hourly_data["wind_speed_180m"] = hourly_wind_speed_180m
hourly_data["wind_direction_120m"] = hourly_wind_direction_120m
hourly_data["wind_direction_180m"] = hourly_wind_direction_180m
hourly_data["temperature_80m"] = hourly_temperature_80m
hourly_data["temperature_120m"] = hourly_temperature_120m
hourly_data["temperature_180m"] = hourly_temperature_180m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)
